In [1]:
import os,sys
project_path = os.path.abspath(os.path.join('..'))
sys.path.append(project_path)
os.environ.get("NEWSAPI_API_KEY")
print("XXXXXXXXXX=>", os.environ.get("NEWSAPI_API_KEY"))
newsapi_api_key = os.environ.get("NEWSAPI_API_KEY")
Connection_String = os.environ.get("NEWSAPI_API_KEY")

XXXXXXXXXX=> e3aa418fa9e249afad49e408b1d3ee40


In [6]:
from src.etl.extract.data_structure_extract_strategy import DataExtractor

config = {
            "type": "api_url",
            "path": "https://newsapi.org/v2/top-headlines",
            "params": {
                "country": "us",
                "pageSize": 2,
                "apiKey": newsapi_api_key,
                "category": "business",
            }
        }
data = DataExtractor.get_extractor(config).extractor()

In [8]:
for item in data.get("articles", []):
            item["category"] = "business"


In [9]:
from src.etl.transform.data_format_strategy import (
    FromJsonToDataFrameFormatter,
    FromDataFrameToCsvFormatter,
    DataFormatter,
    )
from src.etl.transform.data_metadata_strategy import MetadataAppender
from src.etl.transform.data_transform_strategy import RenameColumnsTransform
from datetime import datetime, timedelta, timezone
import pandas as pd


df = DataFormatter(
FromJsonToDataFrameFormatter(array_keys=["articles"])
).formatting(data)
df = RenameColumnsTransform(
{
    "source.id": "source_id",
    "source.name": "source_name",
}
).transform(df)
metadata = {
"createdate": pd.Timestamp.now(tz=timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ"),
"usercreate": "system",
"updatedate": pd.NaT,
"userupdate": "system",
"activedata": True,
"batch_id": "run_id",
"source": "currentsapi",
"layer": "bronze",
}
meta_appender = MetadataAppender(metadata)
df = meta_appender.meta(df)
csv = DataFormatter(
FromDataFrameToCsvFormatter(index=False, encoding="utf-8-sig", sep=",")
).formatting(df)

Renaming columns: {'source.id': 'source_id', 'source.name': 'source_name'}


In [10]:
df.head()

,author,title,description,url,urlToImage,publishedAt,content,category,source_id,source_name,createdate,usercreate,updatedate,userupdate,activedata,batch_id,source,layer
0,None,Dow Jones Futures Rise After Bullish Stock Mar...,The S&P 500 and Nasdaq are still above their 1...,https://www.investors.com/market-trend/stock-m...,https://www.investors.com/wp-content/uploads/2...,2025-10-10T03:00:00Z,Information in Investors Business Daily is for...,business,None,Investor's Business Daily,2025-10-11T06:58:18Z,system,NaT,system,True,run_id,currentsapi,bronze
1,Jasper Goodman,Tensions over crypto bill boil over in the Sen...,A GOP-led effort to pass a landmark crypto bil...,https://www.politico.com/news/2025/10/09/tensi...,https://www.politico.com/dims4/default/resize/...,2025-10-10T02:13:03Z,The clash is the latest twist in a saga over h...,business,politico,Politico,2025-10-11T06:58:18Z,system,NaT,system,True,run_id,currentsapi,bronze


In [5]:
from src.etl.load.data_structure_loader_strategy import PostgresCsvCopyLoader
loader = PostgresCsvCopyLoader(
            Connection_String,
            "bronze",
            "source_news_articles_currentsapi",
        )
loader.create(csv)

{'status': 'success', 'table': 'bronze.source_news_articles_currentsapi'}

In [25]:
csv_cols = ['id','title','body']    # header จากไฟล์
database_table_cols = ['id','title','body','createdate','usercreate','updatedate','userupdate']
target_cols = list(csv_cols)
print("Before Target columns for insertion:", target_cols)
print("----")
for extra in ("body","cat", "createdate", "usercreate", "updatedate", "userupdate"):
    print("extra: ",extra)
    print(f"{extra} in {database_table_cols}: ",extra in database_table_cols)
    print(f"{extra} not in {target_cols}: ",extra not in target_cols)
    
    if (extra in database_table_cols) and (extra not in target_cols):
        print(f"add {extra}")
        target_cols.append(extra)
    print("----")
print("After Target columns for insertion:", target_cols)

Before Target columns for insertion: ['id', 'title', 'body']
----
extra:  body
body in ['id', 'title', 'body', 'createdate', 'usercreate', 'updatedate', 'userupdate']:  True
body not in ['id', 'title', 'body']:  False
----
extra:  cat
cat in ['id', 'title', 'body', 'createdate', 'usercreate', 'updatedate', 'userupdate']:  False
cat not in ['id', 'title', 'body']:  True
----
extra:  createdate
createdate in ['id', 'title', 'body', 'createdate', 'usercreate', 'updatedate', 'userupdate']:  True
createdate not in ['id', 'title', 'body']:  True
add createdate
----
extra:  usercreate
usercreate in ['id', 'title', 'body', 'createdate', 'usercreate', 'updatedate', 'userupdate']:  True
usercreate not in ['id', 'title', 'body', 'createdate']:  True
add usercreate
----
extra:  updatedate
updatedate in ['id', 'title', 'body', 'createdate', 'usercreate', 'updatedate', 'userupdate']:  True
updatedate not in ['id', 'title', 'body', 'createdate', 'usercreate']:  True
add updatedate
----
extra:  userup